In [1]:
import utils
import pandas as pd
import numpy as np
from sklearn.externals import joblib

import category_encoders as ce

import importlib
importlib.reload(utils)

pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

#######################################################################
encoder=joblib.load('/home/gjain/ordinal_encoder.pkl')

# Get data

In [2]:
df=pd.read_csv('/home/gjain/dec_data_000000000000.csv')
# df=pd.read_csv('/home/gjain/jan19_data_000000000000.csv')

df.drop_duplicates(subset='order_number', keep='first', inplace=True)
print (df.shape)

df.target.value_counts()

/home/ubuntu/yes/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,9,30,31,32,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(267688, 91)


0    267449
1    239   
Name: target, dtype: int64

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267688 entries, 0 to 267725
Data columns (total 91 columns):
timestamp                       267688 non-null object
order_number                    267688 non-null object
pay_id                          267688 non-null object
affiliate                       267688 non-null object
channelcode                     205541 non-null object
local_trans_amt                 267688 non-null float64
trans_currency                  267688 non-null object
nonmor                          267688 non-null bool
payment_method                  267688 non-null object
eci                             267688 non-null object
card_cvvmatch                   267688 non-null object
card_avsmatch                   267688 non-null object
card_bin                        157024 non-null float64
card_country                    81136 non-null object
ip_address                      267688 non-null object
customer_email                  267688 non-null object
customer_n

In [37]:
df=df[~pd.isna(df.DepartureDate)]
df.target.value_counts()

0    267449
1    239   
Name: target, dtype: int64

# Feature engineering

In [4]:
email=df['customer_email'].str.split("@",expand=True)
email.columns=['id','domain']

df['email_id']=email['id']
df['email_user_id_len']=email['id'].astype('str').apply(lambda x: len(x) if x is not None else 0)
df['email_domain']=email['domain']

#DOB splits
dob=df['dob'].astype('str').str.split(" ",expand=True)
dob=dob[dob.columns[0]].str.split("-",expand=True)
dob.columns=['year','month','day']
df['dob_year']=dob['year'].astype('str')
df['dob_month']=dob['month'].astype('str')
df['dob_day']=dob['day'].astype('str')

df['bookerIsTravelAgency'] = df.customer_name.astype('str').apply(lambda x: utils.booker_is_travel_agency(x) if x is not None else False)
df['successful_attempt_no']=df.success_attempt_id.apply(lambda x: utils.attempt_no(x))
df['country_ip_flight_MatchScore']=df.apply(utils.country_ip_flight_match_score, axis=1)
df['flight_distance']=df.apply(utils.one_way_distance, axis=1)
df['days_to_departure']=df.apply(utils.days_to_departure,axis=1)
df['vacation_length']=df.apply(utils.vacation_length,axis=1)
df['cities_in_itinerary']=df.Itinerary.astype('str').apply(lambda x: utils.cities_in_itinerary(x) if x is not None else 0)
df['cabin_class']=df.CabinClasses.astype('str').apply(lambda x: utils.cabinclass(x))
df['bookerAge']=df.dob_year.apply(lambda x: utils.get_age(x))
df['name_len']=df.customer_name.astype('str').apply(lambda x: len(x) if x is not None else 0)

## Time related variables to be binned
df['weekend_booking']=df.OrderDateBrandTime_DayOfWeek.apply(lambda x: 1 if x in (1,7) else 0)
df['booking_daytime']= df.OrderDateBrandTime_Hour.apply(lambda x: utils.booking_daytime(x))
df['bookerAgeBracket']=df.bookerAge.apply(lambda x: utils.booker_age_bracket(x))
df['channelcode'].fillna('Direct',inplace=True)

df['dob_month'].fillna(0.0,inplace=True)

df.customer_name.fillna('Unknown',inplace=True)
df.email_id.fillna(value='Unknown',inplace=True)

df['eci']=df['eci'].apply(lambda x: x.replace('0','',1) if isinstance(x,str) else str(x).replace('0','',1))

#This can throw error if customer name or email id is nan
score = np.vectorize(utils.jaro_similarity, otypes=[np.float])(df.customer_name.values, df.email_id.values)
vector_emailed2 = np.vectorize(utils.email_real_check2)
email_sanity_score=vector_emailed2(df.customer_name.values, df.email_id.values, df.dob_year.values, df.dob_month.values,
                                   df.dob_day.values, score)
df['email_sanity_score']=email_sanity_score

#Cyclical features from timestamp
df['ordermonth_sin']=df.OrderDateBrandTime_Month.apply(lambda x: utils.encode_circular(x,"sin", 12))
df['ordermonth_cos']=df.OrderDateBrandTime_Month.apply(lambda x: utils.encode_circular(x,"cos", 12))

dd=df.DepartureDate.apply(lambda x: pd.to_datetime(x).date().month)
df['departuremonth_sin']=dd.apply(lambda x: utils.encode_circular(x,"sin", 12))
df['departuremonth_cos']=dd.apply(lambda x: utils.encode_circular(x,"cos", 12))

######################################################################################
cols_to_drop=['BookerContactPhone','success_attempt_id','email_id','customer_email','dob','card_num','card_country','BookerStreet','BookerZipCode','BookerState','card_bin','customer_name','OriginLatitude','OriginLongitude','DestinationLatitude','DestinationLongitude',
             'DepartureDate','ReturnDate','EndDate','FareType','GdsOfficeId','Itinerary','BookingClasses','CabinClasses','TicketingMethod','GA_DeviceIsMobile','GA_DeviceCategory','OrderDateBrandTime_Year',
             'ip_address','dob_day','dob_year',
#               'OrderDateBrandTime_DayOfWeek',
              'OrderDateBrandTime_Month',
              'OrderDateBrandTime_Hour','bookerAge','BrandTimeZone']

df.drop(cols_to_drop,axis=1,inplace=True)

# Missing value treatment

In [5]:
df.fillna('Unknown',axis=1,inplace=True)

In [7]:
df.shape

(267688, 80)

# Categorical encoding

In [9]:
all_cat_cols=['affiliate','channelcode','trans_currency','nonmor','payment_method','eci','card_cvvmatch','card_avsmatch','customer_title','website_language',
          'brand_continent','HasTicketInsurance','HasInsurance','HasBaggage','HasBaggage_Sale','HasAirhelpPlus','UsedVoucher','HasSP','gender',
          'BookerCity','BookerCountry_Name','airline','Consolidator','OriginAirportCode','OriginCityName','OriginCountryName','OriginWorldPartName','OriginRegion',
          'OriginSubRegion','DestinationAirportCode','DestinationCityName','DestinationCountryName','DestinationWorldPartName','DestinationRegion','DestinationSubRegion',
          'Supplier','DomesticOrInternational','Haul','OneWayOrReturn','IsCombinedOneWay','IsITFare',
          'HasSplitPnr','DeviceType','IsLowCost','email_domain','cabin_class','bookerIsTravelAgency','weekend_booking',
#           'OrderDateBrandTime_Month',
          'dob_month','booking_daytime',
          'bookerAgeBracket',
           #new add
          'OrderDateBrandTime_DayOfWeek',
          #boolean cols
          'nonmor', 'HasTicketInsurance', 'HasInsurance', 'HasBaggage', 'HasBaggage_Sale', 'HasAirhelpPlus', 'UsedVoucher', 'HasSP',
          'IsCombinedOneWay', 'IsITFare', 'HasSplitPnr', 'IsLowCost', 'bookerIsTravelAgency']

encoder.set_params(handle_missing='value')
df = encoder.transform(df)

df.isnull().sum()

TypeError: 'list' object is not callable

In [41]:
df.to_pickle('/home/gjain/dec_data.pkl')